In [2]:
import numpy as np
from scipy import signal

import math

import matplotlib.pyplot as plt
from tqdm import tqdm

%load_ext autoreload
%autoreload 2

TODO:
- check what happens in g_vector for t = 0
- check all code for general mistakes

This notebook simulates the rough Heston model as introduced by Guennoun, Jacquier, Roome and Shi in <cite data-cite="guennoun2018asymptotic">Guennoun, Hamza, et al. "Asymptotic behavior of the fractional Heston model." SIAM Journal on Financial Mathematics 9.3 (2018): 1017-1045.</cite>. In particular it generates a rough volatility. 

In [3]:
n = 1000 # Number of timesteps
sqrtn = math.sqrt(n) 
M = 1 # Number of simulations
rho = 1 # Correlation between the normal distributions dzeta and eta
T = 1 # Total time
alpha = -0.25 # Alpha as in G^\alpha. It has to lie in (-.5, .5) to have any meaning. For negative alphas, you get roughness

# Some volatility parameters
kappa = 1
theta = 10 # 
vola_0 = 10 # initial value of the non-rough vola
eta = 1
ksi = 1
X_0 = 1 # initial value of X, the log-stock price 

In [4]:
def g(t):
    return t**alpha
g_vector = np.fromfunction(lambda i: g(T*i/n), shape=(n,), dtype=int)
g_vector[0] = 0

<ipython-input-4-096511ec5bcc>:2: RuntimeWarning: divide by zero encountered in power
  return t**alpha


In [5]:
def b(y):
    return kappa * (theta-y)
def a(y):
    return ksi * math.sqrt(y)
def Phi(y):
    return eta + y

The Notebook first simulates the volatility according to
$$
dV_t = \kappa(\theta - V_t)dt + \xi \sqrt{V_t}dW_t
$$

In [6]:
vola = np.zeros((n,)) # The discretised non-rough volatility process
Delta_vola = np.zeros((n,)) # vola_i - vola_{i-1}

In [8]:
W = np.random.normal(0.0, 1.0, size=(n,)) # The unscaled brownian motion
vola[0] = vola_0
for i in range(1, n):
    vola[i] = vola[i-1] + (T/n)*b(vola[i-1]) + (T/sqrtn)*a(vola[i-1])*W[i-1]
    Delta_vola[i] = vola[i] - vola[i - 1]

Using Fast Fourier Transform (FFT), the volatility is made rough. For more details, see <cite data-cite="horvath2017functional">Horvath, Blanka, Antoine Jack Jacquier, and Aitor Muguruza. "Functional central limit theorems for rough volatility." Available at SSRN 3078743 (2017)</cite>

In [23]:
# For some unknown reason, this doesn't work, but a valid solution was found
fourier_transformed_G_alpha = np.fft.fft(g_vector)*np.fft.fft(Delta_vola) 
G_alpha_Y = np.fft.ifft(fourier_transformed_G_alpha) # This is supposed to the the rough volatility 
G_alpha_Y[:5]

array([7.39563798-4.19362384e-16j, 7.51689192-4.38617970e-16j,
       7.20257469+3.05391207e-16j, 6.37144881-1.57953589e-16j,
       6.53256048-4.28244168e-16j])

In [22]:
G_alpha_Y[:5]

array([7.39563798-4.19362384e-16j, 7.51689192-4.38617970e-16j,
       7.20257469+3.05391207e-16j, 6.37144881-1.57953589e-16j,
       6.53256048-4.28244168e-16j])

In [16]:
# To compare it with the theoretical value
G_a_Y = np.zeros((n,))
for i in range(n):
    for k in range(1, i+1):
        G_a_Y[i] += g_vector[i-k + 1] * Delta_vola[k] 
G_a_Y[10:]

array([-1.89326073, -1.9502175 , -1.48756089, -0.84732363, -1.34611327,
       -0.28689178, -0.09705382,  1.00839649,  0.56628154,  0.79304259,
        0.13006921,  0.8368511 , -0.19247409,  0.34186758,  0.23501829,
        1.28597314,  0.68626395,  0.72927482, -0.03101468, -0.51092152,
       -0.59315598,  0.15264893,  0.75119205,  1.23181771,  1.43659125,
        1.30812338,  2.37135006,  2.26065809,  3.15824966,  2.52794852,
        2.1004175 ,  2.81407865,  2.92733226,  3.61919309,  4.94320257,
        3.65088622,  4.41239186,  4.15077382,  4.73246178,  4.62134321,
        5.7408667 ,  5.85199245,  5.83924496,  6.27131554,  5.90293133,
        6.64348994,  7.13868181,  6.77398978,  6.18111302,  6.65916552,
        5.71974237,  5.42381041,  4.90883144,  4.69533158,  4.5287812 ,
        4.25442289,  4.85965194,  5.90268314,  6.44966766,  5.50009939,
        6.92166892,  7.68106954,  6.59237065,  6.40923914,  6.56995859,
        6.39854855,  7.07207269,  6.99663322,  5.7872634 ,  6.44

In [ ]:
rough_vola = signal.fftconvolve(g_vector, Delta_vola, 'same')

<!--bibtex

@article{guennoun2018asymptotic,
  title={Asymptotic behavior of the fractional Heston model},
  author={Guennoun, Hamza and Jacquier, Antoine and Roome, Patrick and Shi, Fangwei},
  journal={SIAM Journal on Financial Mathematics},
  volume={9},
  number={3},
  pages={1017--1045},
  year={2018},
  publisher={SIAM}
}

@article{horvath2017functional,
  title={Functional central limit theorems for rough volatility},
  author={Horvath, Blanka and Jacquier, Antoine Jack and Muguruza, Aitor},
  journal={Available at SSRN 3078743},
  year={2017}
}

-->

Using this rough volatility, a standard simulation is run on the log prices, based on the equation

$$
dX_t = -\frac{1}{2}V_tdt + \sqrt{V_t} dB_t
$$

In [ ]:
X = np.zeros((M, n))
for j in range(M):
    # Create correlated Brownian motion
    randoms = np.random.normal(0.0, 1.0, size=(n,))
    B = rho*W + math.sqrt(1 - rho**2) * randoms
    X[j, 0] = X_0
    for i in range(1, n):
        X[j, i] = X[j, i-1] - (1/2)*(T/n)*Phi(rough_vola[i-1]) + math.sqrt((T/n) * Phi(rough_vola[i-1])) * B[i-1]

In [ ]:
# A function which gives the brownian motion used to generate the rough volatility and the volatility.
def rough_volatility():
    vola = np.zeros((n,))
    Delta_vola = np.zeros((n,))
    W = np.random.normal(0.0, 1.0, size=(n,))
    g_vector = np.fromfunction(lambda i: g(T*i/n), shape=(n,), dtype=int)
    vola[0] = vola_0
    for i in range(1, n):
        vola[i] = vola[i-1] + (T/n)*b(vola[i-1]) + (T/sqrtn)*a(vola[i-1])*W[i-1]
        Delta_vola[i] = vola[i] - vola[i - 1]
    rough_vola = signal.fftconvolve(g_vector, Delta_vola, 'same')
    return (W, G_alpha_Y)

In [ ]:
print(X)

In [ ]:
plt.plot(rough_vola)